In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import os
import json
from glob import glob

import tensorflow as tf
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

import ptflops
import torch

In [ ]:
BASE_EXP_PATH = "evalPaper/experiments"

In [ ]:
def get_keras_flops(model):
    forward_pass = tf.function(model.call, input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])
    graph_info = profile(forward_pass.get_concrete_function().graph, options=ProfileOptionBuilder.float_operation())
    flops = graph_info.total_float_ops
    return flops

In [ ]:
input_shape_map = {
    "skodar": (1, 64, 30)
}


def get_experiment_data(model = None, dataset = None):
    subdirs = [d for d in os.listdir(BASE_EXP_PATH) if os.path.isdir(os.path.join(BASE_EXP_PATH, d))]
    subdirs = [os.path.join(BASE_EXP_PATH, d) for d in subdirs]

    all_exps = []
    for subdir in subdirs:
        with open(os.path.join(subdir, "args.json"), "r") as f:
            args = json.load(f)
            exp_subdirs = [d for d in os.listdir(subdir) if os.path.isdir(os.path.join(subdir, d))]
            args["experiments"] = []
            for exp_subdir in exp_subdirs:
                with open(os.path.join(subdir, exp_subdir, "result.json"), "r") as f:
                    results = json.load(f)

                base_path = os.path.join(subdir, exp_subdir, "model*.")
                model_files = glob(base_path + "h5") + glob(base_path + "pth")
                assert len(model_files) == 1
                model_file = model_files[0]


                if model_file.endswith(".h5"):
                    pass
                    keras_model = tf.keras.models.load_model(model_file)
                    keras_flops = get_keras_flops(keras_model)
                    results["flops"] = keras_flops
                    print("KERAS_FLOPS", keras_flops)

                if model_file.endswith(".pth"):
                    # Load pytorch file:
                    torch_model = torch.load(model_file)
                    torch_input_shape = input_shape_map[args["dataset"]]
                    results["flops"] = float(ptflops.get_model_complexity_info(torch_model, torch_input_shape, verbose=True, flops_units="KMac")[0].split(" ")[0
                    ]) * 1000
                args["experiments"].append(results)
            all_exps.append(args)

    if model is not None:
        all_exps = [exp for exp in all_exps if exp["model"] == model]

    if dataset is not None:
        all_exps = [exp for exp in all_exps if exp["dataset"] == dataset]

    return all_exps

In [ ]:
exps = get_experiment_data(None, None)
print(exps)
# acc_vali = exps[0]["experiments"][0]["eval"]["acc_vali"]
# print(acc_vali)

In [203]:
for exp in exps:
    print(exp["model"], exp["dataset"])
    for e in exp["experiments"]:
        print(e["flops"])

fftmixer skodar
984610.0
micronas skodar
16449330
37720626
23044914
14532658
33334322
9679410
21229490
60008242
21131570
64337714
48361650
50676274
38586162
33402290
17219506
32205106
44183986
44904754
23428146
48643122
tinyhar skodar
9044540.0
